In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
from transformers import pipeline
from fpdf import FPDF
import os
from IPython.display import Image, display

# Initialize VADER sentiment analyzer for English
analyzer = SentimentIntensityAnalyzer()

# Initialize transformers pipelines
sentiment_model = pipeline("sentiment-analysis")
summarizer = pipeline("summarization")

# Ensure reproducibility of language detection
DetectorFactory.seed = 0

# Expanded emotion mapping for English, Hindi, and Marathi using compound score
def get_emotion(compound_score, lang):
    if lang == 'en':  # For English
        if compound_score >= 0.9:
            return 'joy'
        elif 0.7 <= compound_score < 0.9:
            return 'love'
        elif 0.5 <= compound_score < 0.7:
            return 'excitement'
        elif 0.3 <= compound_score < 0.5:
            return 'anticipation'
        elif 0 < compound_score < 0.3:
            return 'contentment'
        elif -0.3 < compound_score <= 0:
            return 'fear'
        elif -0.5 < compound_score <= -0.3:
            return 'sadness'
        elif -0.7 < compound_score <= -0.5:
            return 'anger'
        elif compound_score <= -0.7:
            return 'disgust'
        else:
            return 'neutral'

    elif lang == 'hi':  # For Hindi
        if compound_score >= 0.9:
            return 'khushi (joy)'
        elif 0.7 <= compound_score < 0.9:
            return 'prem (love)'
        elif 0.5 <= compound_score < 0.7:
            return 'utsah (excitement)'
        elif 0.3 <= compound_score < 0.5:
            return 'abhimaan (pride)'
        elif 0 < compound_score < 0.3:
            return 'asha (hope)'
        elif -0.3 < compound_score <= 0:
            return 'dar (fear)'
        elif -0.5 < compound_score <= -0.3:
            return 'dukhi (sadness)'
        elif -0.7 < compound_score <= -0.5:
            return 'gussa (anger)'
        elif compound_score <= -0.7:
            return 'nirasha (despair)'
        else:
            return 'neutral'

    elif lang == 'mr':  # For Marathi
        if compound_score >= 0.9:
            return 'khushi (खुशी - joy)'
        elif 0.7 <= compound_score < 0.9:
            return 'prem (प्रेम - love)'
        elif 0.5 <= compound_score < 0.7:
            return 'utsah (उत्साह - excitement)'
        elif 0.3 <= compound_score < 0.5:
            return 'abhimaan (अभिमान - pride)'
        elif 0 < compound_score < 0.3:
            return 'samadhan (समाधान - contentment)'
        elif -0.3 < compound_score <= 0:
            return 'dar (भय - fear)'
        elif -0.5 < compound_score <= -0.3:
            return 'dukhi (दुःख - sadness)'
        elif -0.7 < compound_score <= -0.5:
            return 'rag (राग - anger)'
        elif compound_score <= -0.7:
            return 'ashakti (अशक्ती - helplessness)'
        else:
            return 'neutral'

def analyze_text(text, lang):
    # Split text into sentences
    sentences = [s.strip() for s in text.split('.') if s.strip()]
    sentence_emotions = {}
    detailed_sentiments = []

    for sentence in sentences:
        if lang == 'en':
            sentiment = analyzer.polarity_scores(sentence)
            compound_score = sentiment['compound']
        else:
            try:
                sentiment_result = sentiment_model(sentence)[0]
                compound_score = sentiment_result['score']
            except Exception as e:
                print(f"Error analyzing sentiment for sentence: {sentence}\n{e}")
                compound_score = 0  # Default to neutral if error occurs

        emotion = get_emotion(compound_score, lang)
        
        detailed_sentiments.append({
            'sentence': sentence,
            'compound_score': compound_score,
            'emotion': emotion
        })

        if emotion in sentence_emotions:
            sentence_emotions[emotion] += 1
        else:
            sentence_emotions[emotion] = 1

    return sentence_emotions, detailed_sentiments

def plot_emotions_interactive(emotions, chart_type, output_path):
    labels = list(emotions.keys())
    sizes = list(emotions.values())

    if chart_type == 'pie':
        fig = go.Figure(data=[go.Pie(labels=labels, values=sizes, hole=0.3)])
        fig.update_layout(title_text='Emotion Analysis')
    elif chart_type == 'bar':
        fig = go.Figure(data=[go.Bar(x=labels, y=sizes)])
        fig.update_layout(title_text='Emotion Analysis', xaxis_title='Emotions', yaxis_title='Count')
    
    # Save the figure as a static image
    fig.write_image(output_path)
    print(f"Emotion graph saved as {output_path}")

    # Display the image in Jupyter Notebook
    display(Image(filename=output_path))

def summarize_text(text):
    summary = summarizer(text, max_length=150, min_length=30, do_sample=False)
    return summary[0]['summary_text']

def generate_pdf(summary, detailed_sentiments, chart_image_path, output_pdf_path):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    
    # Title
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, "Sentiment Analysis Report", ln=True, align='C')
    
    pdf.ln(10)  # Add a line break
    
    # Summary Section
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(0, 10, "Summary of the Text:", ln=True)
    pdf.set_font("Arial", '', 12)
    pdf.multi_cell(0, 10, summary)
    
    pdf.ln(5)
    
    # Detailed Sentiment Breakdown
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(0, 10, "Detailed Sentiment Breakdown:", ln=True)
    pdf.set_font("Arial", '', 12)
    
    for item in detailed_sentiments:
        sentence = item['sentence']
        emotion = item['emotion']
        compound = item['compound_score']
        pdf.multi_cell(0, 10, f"Sentence: {sentence}")
        pdf.multi_cell(0, 10, f"Emotion: {emotion}")
        pdf.multi_cell(0, 10, f"Compound Score: {compound}")
        pdf.ln(2)
    
    pdf.ln(5)
    
    # Emotion Graph
    if os.path.exists(chart_image_path):
        pdf.set_font("Arial", 'B', 12)
        pdf.cell(0, 10, "Emotion Analysis Graph:", ln=True)
        pdf.image(chart_image_path, x=None, y=None, w=170)  # Adjust width as needed
    else:
        pdf.set_font("Arial", '', 12)
        pdf.cell(0, 10, "Emotion graph image not found.", ln=True)
    
    # Save the PDF
    pdf.output(output_pdf_path)
    print(f"Report saved as {output_pdf_path}")

def main():
    text = input("Enter your paragraph (English, Hindi, or Marathi): ").strip()

    if not text:
        print("Text is empty. Please enter a valid paragraph.")
        return

    try:
        lang = detect(text)
        if lang not in ['en', 'hi', 'mr']:
            print("Unsupported language. Please enter text in English, Hindi, or Marathi.")
            return
    except LangDetectException as e:
        print(f"Language detection error: {e}")
        return

    # Preprocess text to handle potential issues
    text = text.replace('\u200B', '')  # Remove zero-width spaces
    text = ' '.join(text.split())  # Remove extra spaces

    emotions, detailed_sentiments = analyze_text(text, lang)
    summary = summarize_text(text)
    
    print("\nSummary of the text:")
    print(summary)
    
    print("\nDetailed Sentiment Breakdown:")
    for item in detailed_sentiments:
        print(f"Sentence: {item['sentence']}")
        print(f"Emotion: {item['emotion']}")
        print(f"Compound Score: {item['compound_score']}")
        print()

    chart_type = input("Enter chart type (bar/pie): ").strip().lower()
    
    if chart_type not in ['bar', 'pie']:
        print("Unsupported chart type. Defaulting to pie chart.")
        chart_type = 'pie'
    
    # Define paths for saving the chart and PDF
    chart_image_path = "emotion_chart.png"
    output_pdf_path = "sentiment_analysis_report.pdf"
    
    plot_emotions_interactive(emotions, chart_type, chart_image_path)
    
    if lang == 'en':  # PDF option only available for English
        save_pdf = input("Do you want to save the PDF report? (yes/no): ").strip().lower()
        if save_pdf == 'yes':
            # Ensure unique PDF filename
            base_filename = "sentiment_analysis_report"
            counter = 1
            while os.path.exists(f"{base_filename}({counter}).pdf"):
                counter += 1
            output_pdf_path = f"{base_filename}({counter}).pdf"

            generate_pdf(summary, detailed_sentiments, chart_image_path, output_pdf_path)
    else:
        print("PDF report option is only available for English text.")

if __name__ == "__main__":
    main()
